In [1]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

<b>Example: automobile data</b>

https://www.kaggle.com/toramky/automobile-dataset

In [2]:
carsdata=pd.read_csv('../data/Automobile_data.csv')
carsdata.sample(5)

symboling normalized-losses         make fuel-type aspiration  \
143          0               102       subaru       gas        std   
2            1                 ?  alfa-romero       gas        std   
160          0                91       toyota       gas        std   
125          3               186      porsche       gas        std   
44           1                 ?        isuzu       gas        std   

    num-of-doors body-style drive-wheels engine-location  wheel-base  ...  \
143         four      sedan          fwd           front        97.2  ...   
2            two  hatchback          rwd           front        94.5  ...   
160         four      sedan          fwd           front        95.7  ...   
125          two  hatchback          rwd           front        94.5  ...   
44           two      sedan          fwd           front        94.5  ...   

     engine-size  fuel-system  bore  stroke compression-ratio horsepower  \
143          108         mpfi  3.62    2.64               9.0         94   
2            152         mpfi  2.68    3.47               9.0        154   
160           98         2bbl  3.19    3.03               9.0         70   
125          151         mpfi  3.94    3.11               9.5        143   
44            90         2bbl  3.03    3.11               9.6         70   

     peak-rpm city-mpg highway-mpg  price  
143      5200       26          32   9960  
2        5000       19          26  16500  
160      4800       38          47   7738  
125      5500       19          27  22018  
44       5400       38          43      ?  

[5 rows x 26 columns]

To fit a model to find important prefictors of `price`, let's start with some predictors to fit an initial model.

In [3]:
cars=carsdata[['engine-size','horsepower','city-mpg','price']].copy()
#names have been giving me trouble..
cars.columns = ['enginesize', 'horsepower', 'citympg', 'price']


In [4]:
#deal with missing values: the missing values here are "?"
cars['enginesize'] = cars['enginesize'].replace('?', np.nan)
cars['horsepower'] = cars['horsepower'].replace('?', np.nan)
cars['citympg']   = cars['citympg'].replace('?', np.nan)
cars['price']     = cars['price'].replace('?', np.nan)
cars.isnull().sum()

enginesize    0
horsepower    2
citympg       0
price         4
dtype: int64

In [5]:
#Not too many missing values, for now let's drop them
#Recall how we could impute missing values from EDA
carsnew=cars.dropna().copy()
carsnew.info()
carsnew.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 0 to 204
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   enginesize  199 non-null    int64 
 1   horsepower  199 non-null    object
 2   citympg     199 non-null    int64 
 3   price       199 non-null    object
dtypes: int64(2), object(2)
memory usage: 7.8+ KB


enginesize    0
horsepower    0
citympg       0
price         0
dtype: int64

In [6]:
#one more problem: price is a object, not int64(numeric), so is horsepower!
carsnew['horsepower'] = carsnew['horsepower'].astype('int64')
carsnew['price'] = carsnew['price'].astype('int64')
carsnew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 0 to 204
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   enginesize  199 non-null    int64
 1   horsepower  199 non-null    int64
 2   citympg     199 non-null    int64
 3   price       199 non-null    int64
dtypes: int64(4)
memory usage: 7.8 KB


Now we can fit mlr: price~enginesize + citympg+ horsepower

In [7]:
reg = smf.ols('price~enginesize+citympg+horsepower',data=carsnew).fit()

#print(dir(reg)) #members of the object provided by the modelling

In [8]:
#Model summary
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     251.2
Date:                Mon, 15 Sep 2025   Prob (F-statistic):           1.03e-66
Time:                        11:07:50   Log-Likelihood:                -1912.4
No. Observations:                 199   AIC:                             3833.
Df Residuals:                     195   BIC:                             3846.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2547.0951   2914.668     -0.874      0.383   -8295.415    3201.225
enginesize   124.3388     10.951     11.355      0.000     102.742     145.935
citympg     -151.3184     70.849     -2.136      0.034    -291.047     -11.590
horsepower    37.0876     16.262      2.281      0.024       5.016      69.159
==============================================================================
Omnibus:                       10.990   Durbin-Watson:                   0.771
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               17.130
Skew:                           0.312   Prob(JB):                     0.000191
Kurtosis:                       4.295   Cond. No.                     1.96e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.96e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<b>Anova F test</b>

- from global anova: the model is significant compared to the null model

- from t test: enginesize, citympg, horsepower are all significant predictors of price

But recall that when we extract `anova_lm` seprately, different `typ` can give us the results of either <b>sequential F test</b> results or <b> partial F test </b> results. 

In [9]:
# typ=1 and typ=2 in anova_lm gives different values in SS and F test
sm.stats.anova_lm(reg, typ=1) #sequential

df        sum_sq       mean_sq           F        PR(>F)
enginesize    1.0  9.625888e+09  9.625888e+09  724.441229  1.387785e-67
citympg       1.0  3.186064e+08  3.186064e+08   23.978216  2.039654e-06
horsepower    1.0  6.911228e+07  6.911228e+07    5.201368  2.365037e-02
Residual    195.0  2.591029e+09  1.328733e+07         NaN           NaN

In [10]:
sm.stats.anova_lm(reg, typ=2) #partial

#h0: reduced model with the specific predictor dropped 
#h1: always the full model 

sum_sq     df           F        PR(>F)
enginesize  1.713088e+09    1.0  128.926426  2.904006e-23
citympg     6.061127e+07    1.0    4.561585  3.394397e-02
horsepower  6.911228e+07    1.0    5.201368  2.365037e-02
Residual    2.591029e+09  195.0         NaN           NaN

In [11]:
# typ-3 is the same as typ=2, just added a test on intercept
sm.stats.anova_lm(reg, typ=3)

sum_sq     df           F        PR(>F)
Intercept   1.014728e+07    1.0    0.763681  3.832538e-01
enginesize  1.713088e+09    1.0  128.926426  2.904006e-23
citympg     6.061127e+07    1.0    4.561585  3.394397e-02
horsepower  6.911228e+07    1.0    5.201368  2.365037e-02
Residual    2.591029e+09  195.0         NaN           NaN

## Typ=1

In [12]:
#Take a closer look at typ=1: reduced model without citympg
reg_reduced_1 = smf.ols('price~enginesize',data=carsnew).fit()
sm.stats.anova_lm(reg_reduced_1, typ=1)

df        sum_sq       mean_sq           F        PR(>F)
enginesize    1.0  9.625888e+09  9.625888e+09  636.609809  1.265067e-63
Residual    197.0  2.978748e+09  1.512055e+07         NaN           NaN

In [13]:
#Take a closer look at typ=1: full model with citympg
reg_full_1 = smf.ols('price~enginesize+citympg',data=carsnew).fit()
sm.stats.anova_lm(reg_full_1, typ=1)

df        sum_sq       mean_sq           F        PR(>F)
enginesize    1.0  9.625888e+09  9.625888e+09  709.238315  4.852522e-67
citympg       1.0  3.186064e+08  3.186064e+08   23.475016  2.566218e-06
Residual    196.0  2.660141e+09  1.357215e+07         NaN           NaN

In [14]:
## SS(Resid; enginesize only) - SS(Resid; enginesize+citympg) = 
2.978748e+09 - 2.660141e+09

318607000.0

The difference in SSE is the same as the SS for citympg in y~enginesize+citympg+horsepower

In [15]:
sm.stats.anova_lm(reg, typ=1)

df        sum_sq       mean_sq           F        PR(>F)
enginesize    1.0  9.625888e+09  9.625888e+09  724.441229  1.387785e-67
citympg       1.0  3.186064e+08  3.186064e+08   23.978216  2.039654e-06
horsepower    1.0  6.911228e+07  6.911228e+07    5.201368  2.365037e-02
Residual    195.0  2.591029e+09  1.328733e+07         NaN           NaN

But, changing the predictor order in typ=1 alters the result because you are comparing different reduced and full models now

In [16]:
reg_order= smf.ols('price~citympg+horsepower+enginesize',data=carsnew).fit()
sm.stats.anova_lm(reg_order, typ=1)

df        sum_sq       mean_sq           F        PR(>F)
citympg       1.0  5.988084e+09  5.988084e+09  450.661312  1.369003e-52
horsepower    1.0  2.312434e+09  2.312434e+09  174.033076  8.065939e-29
enginesize    1.0  1.713088e+09  1.713088e+09  128.926426  2.904006e-23
Residual    195.0  2.591029e+09  1.328733e+07         NaN           NaN

<b> From sequential F tests </b>

- We can kind of "rank" the importance of the predictors. 
- Since in the above example all predictors are significant, we will leave this for the credit data example. 

## Typ=2

In [17]:
#Take a closer look at typ=2: reduced model without citympg
reg_reduced_2 = smf.ols('price~enginesize+horsepower',data=carsnew).fit()
sm.stats.anova_lm(reg_reduced_2, typ=2)


sum_sq     df           F        PR(>F)
enginesize  1.672205e+09    1.0  123.603577  1.403756e-22
horsepower  3.271074e+08    1.0   24.178641  1.852652e-06
Residual    2.651640e+09  196.0         NaN           NaN

In [18]:
#full
sm.stats.anova_lm(reg, typ=2)

sum_sq     df           F        PR(>F)
enginesize  1.713088e+09    1.0  128.926426  2.904006e-23
citympg     6.061127e+07    1.0    4.561585  3.394397e-02
horsepower  6.911228e+07    1.0    5.201368  2.365037e-02
Residual    2.591029e+09  195.0         NaN           NaN

In [19]:
#citympg	6.061127e+07	 = Residual(red) 2.651640e+09 - Residual(full)	2.591029e+09	
2.651640e+09 - 2.591029e+09 #approx bc of sigfig issue

60611000.0

The difference in SSE (SS Residual) between reduced and full models is the SS of citympg in the full model!!

Similar story if we check other reduced models with dropped engine or horsepower:

In [20]:
reg_reduced_noengine = smf.ols('price~citympg+horsepower',data=carsnew).fit()
sm.stats.anova_lm(reg_reduced_noengine, typ=2)

sum_sq     df           F        PR(>F)
citympg     1.972876e+07    1.0    0.898404  3.443771e-01
horsepower  2.312434e+09    1.0  105.303180  4.767363e-20
Residual    4.304117e+09  196.0         NaN           NaN

In [21]:
4.304117e+09 - 2.591029e+09 #enginesize	1.713088e+09	
# = SS(engine) in full model

1713088000.0

In [22]:
reg_reduced_nohorse = smf.ols('price~citympg+enginesize',data=carsnew).fit()
sm.stats.anova_lm(reg_reduced_nohorse, typ=2)

sum_sq     df           F        PR(>F)
citympg     3.186064e+08    1.0   23.475016  2.566218e-06
enginesize  3.956410e+09    1.0  291.509467  1.213994e-40
Residual    2.660141e+09  196.0         NaN           NaN

In [23]:
2.660141e+09 - 2.591029e+09  #horsepower	~6.911228e+07
# = SS(horsepower) in full model

69112000.0

<b> From partial F tests </b>

- Every reduced model is compared to the same full model:

For example, the F test for `enginesize` is comparing 

- Reduced model: price ~ citympg+ horsepower
- Full model: price ~ enginesize+ citympg+ horsepower

So it will suggest the significance of one predictor conditional on the fact ALL OTHER predictors are already in the model. We can use it to 

- judge the "conditional" importance of each predictor
- pick a reduced model with one less predictor that's better than the full model. 

<b>Example: Credit data</b>

In [31]:
# example: Credit data
credit = pd.read_csv("../data/Credit.csv")
credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  400 non-null    int64  
 1   Income      400 non-null    float64
 2   Limit       400 non-null    int64  
 3   Rating      400 non-null    int64  
 4   Cards       400 non-null    int64  
 5   Age         400 non-null    int64  
 6   Education   400 non-null    int64  
 7   Gender      400 non-null    object 
 8   Student     400 non-null    object 
 9   Married     400 non-null    object 
 10  Ethnicity   400 non-null    object 
 11  Balance     400 non-null    int64  
dtypes: float64(1), int64(7), object(4)
memory usage: 37.6+ KB


In [25]:
credit['Income'] = pd.to_numeric(credit['Income'])

In [26]:
model =smf.ols('Balance~Income + Limit +Rating + Age',data=credit).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.877
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     705.6
Date:                Mon, 15 Sep 2025   Prob (F-statistic):          2.16e-178
Time:                        11:07:56   Log-Likelihood:                -2599.9
No. Observations:                 400   AIC:                             5210.
Df Residuals:                     395   BIC:                             5230.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -445.1048     40.576    -10.970      0.000    -524.877    -365.332
Income        -7.6127      0.382    -19.945      0.000      -8.363      -6.862
Limit          0.0818      0.045      1.834      0.067      -0.006       0.170
Rating         2.7314      0.664      4.111      0.000       1.425       4.038
Age           -0.8561      0.478     -1.789      0.074      -1.797       0.084
==============================================================================
Omnibus:                       94.733   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              165.919
Skew:                           1.374   Prob(JB):                     9.36e-37
Kurtosis:                       4.550   Cond. No.                     2.65e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.65e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

From individual t-testing, Income and Rating are significant, while Limit is on the boundary. So t-tests suggests the model: y~Income+Rating

In [27]:
# try sequential anova with the same order

sm.stats.anova_lm(model, typ=1)



df        sum_sq       mean_sq            F         PR(>F)
Income      1.0  1.813117e+07  1.813117e+07   691.691426   7.902872e-89
Limit       1.0  5.533791e+07  5.533791e+07  2111.102870  1.465723e-160
Rating      1.0  4.328357e+05  4.328357e+05    16.512381   5.832806e-05
Age         1.0  8.394134e+04  8.394134e+04     3.202304   7.430058e-02
Residual  395.0  1.035406e+07  2.621280e+04          NaN            NaN

Here:
- p-values disagree with t tests
- Suggests y\~Income+Limit is sig. better than just y\~Income, and  y\~Income+Limit+Rating is sig. better than y\~Income+Limit
- y~ Income+Limit+ Rating is the best model according to seq. F-test (since adding age is not significant; 7.430058e-02>0.05)

In [34]:
#Now consider putting the questionable one (Limit) at the end 
model_2 =smf.ols('Balance~Age + Income + Rating +Limit',data=credit).fit()
sm.stats.anova_lm(model_2, typ=1)

df        sum_sq       mean_sq            F         PR(>F)
Age         1.0  2.840283e+02  2.840283e+02     0.010835   9.171479e-01
Income      1.0  1.868031e+07  1.868031e+07   712.640837   1.809916e-90
Rating      1.0  5.521706e+07  5.521706e+07  2106.492535  2.108937e-160
Limit       1.0  8.820013e+04  8.820013e+04     3.364774   6.735675e-02
Residual  395.0  1.035406e+07  2.621280e+04          NaN            NaN

Now the sequential anova suggests y~Income+Rating might be enough, adding Limit doesn't significantly improve it anymore.

- limit might be on the boundary

<b>Can you try typ=2?</b> What does that tell us?

For the candidates, let's look at R^2 and adj-R^2

In [36]:
model_c1 = smf.ols('Balance~Income + Rating ',data=credit).fit() #from t test and anova(typ=2)
model_c2 = smf.ols('Balance~Income + Rating +Limit',data=credit).fit()#from anova(typ=1)
model_c3 = smf.ols('Balance~Income + Rating +Age',data=credit).fit() #not that strong
model_c4 = smf.ols('Balance~Income + Rating +Age + Limit',data=credit).fit() #not that strong

In [37]:
print(model_c1.rsquared, model_c1.rsquared_adj)
print(model_c2.rsquared, model_c2.rsquared_adj)
print(model_c3.rsquared, model_c3.rsquared_adj)
print(model_c4.rsquared, model_c4.rsquared_adj)


0.8751179476994355 0.8744888189724805
0.8762389456262863 0.8753013618810309
0.8761884500989228 0.8752504838117934
0.8772342199048692 0.875991022131754


Out of possible candidates , based on adjusted R^2, model_c2 is the best choice;

the analysis during the way suggests the possible ranking of importance:
Rating ~ Income ~ Rating > Limit > Age 

### Remark 

You can already see that different methods lead to different conclusions. After we learn modeling diagnosis and more selection methods, we will consider many model selection criteria, and argue to make a decision based on a specific one which makes sense in terms of interpretation and prediction for the problem at hand; if still hard to decide, you can try the prediction on a test set to go with the one with better prediction performance. 

